In [66]:
import matplotlib.pyplot as plt; import seaborn as sns; import numpy as np; import pandas as pd; import os
import glob
%matplotlib inline  

In [67]:
analysis_home = ('/user_data/kbond/fmriprep_confound_denoise')
data_path = ('/lab_data/coaxlab/Projects/loki_1/fmriprep_BOLD_data/')

In [68]:
confound_df = pd.read_csv(os.path.join(analysis_home, 'basic_confound_df.csv'))

In [69]:
confound_df.head()

,a_comp_cor_00,a_comp_cor_01,a_comp_cor_02,a_comp_cor_03,a_comp_cor_04,a_comp_cor_05,rot_x,rot_y,rot_z,run,session,slice,sub,trans_x,trans_y,trans_z
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000235,-8.894170e-05,0.000000,5,5,0,811,0.009999,-0.019949,0.032686
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000093,-0.000000e+00,0.000000,5,5,1,811,0.000000,0.006128,-0.020904
2,0.021215,0.039827,-0.039051,0.038939,-0.061366,0.015082,-0.000093,-1.155650e-24,-0.000085,5,5,2,811,0.006466,0.009448,-0.038227
3,0.054664,0.023060,-0.002183,0.025691,-0.046342,0.067015,-0.000137,1.589700e-04,0.000000,5,5,3,811,0.002176,0.165384,0.009432
4,0.013892,0.047422,-0.029474,0.011404,-0.050552,0.074103,0.000025,8.089280e-05,0.000000,5,5,4,811,0.000855,-0.027810,-0.016503


In [93]:
a_comp_cor_cols = [col for col in confound_df if col.startswith('a_comp_cor')]

# Key confounds
## Estimated head-motion parameters (the 6 rigid-body motion parameters (3 translations and 3 rotations) estimated relative to a reference image)):
* trans_x
* trans_y
* trans_z
* rot_x
* rot_y
* rot_z 

## Global signals:
* csf: the average signal within the anatomically-derived eroded CSF mask
* white_matter:  the average signal within the anatomically-derived eroded WM masks
* global_signal: the average signal within the brain mask

* **a_comp_cor**: principal components of the above anatomical signals 

## Derivatives of head motion parameters & derivatives of their quadratic expansion 


In [18]:
# !pip install nipype --user
# !pip install niworkflows --user

In [13]:
from niworkflows.interfaces.confounds import temporal_derivatives, exponential_terms

In [70]:
# Formula for generating model expansions. By default, the
# 32-parameter expansion will be generated. Note that any expressions
# to be expanded *must* be in parentheses, even if they include only
# a single variable (e.g., ``(x)^2``, not ``x^2``). Examples:
# * ``rps + wm + csf + gsr`` : 9-parameter model. ``rps`` denotes realignment
#   parameters, ``wm`` denotes mean white matter signal, ``csf`` denotes mean
#   cerebrospinal fluid signal, and ``gsr`` denotes mean global signal.
# * ``(dd1(rps + wm + csf + gsr))^^2`` : 36-parameter expansion.
#   ``rps + wm + csf + gsr`` denotes that realignment parameters and mean
#   WM, CSF, and global signals should be included. ``dd1`` denotes that
#   these signals should be augmented with their first temporal
#   derivatives. ``^^2`` denotes that the original signals and temporal
#   derivatives should be augmented with quadratic expansions.
# * ``(dd1(rps))^^2`` : 24-parameter expansion. ``rps`` denotes that
#   realignment parameters should be included. ``dd1`` and ``^^2`` denote
#   temporal derivative and quadratic expansions as above.
# * ``(dd1(rps + wm + csf + gsr))^^2 + others`` : generate all expansion
#   terms necessary for a 36-parameter model as above, and
#   concatenate those expansion terms to all other regressor columns
#   in the confounds file.

model_formula='(dd1(rps))^^2' # 24-parameter expansion

In [81]:
temporal_derivatives_order = [0,1]
exponential_expansion_order = [1,2]

# calculate derivatives for head-motion estimates 
# (trans_ and rot_) and 
# global signals (white_matter, csf, and global_signal)

# original signals & derivatives should have exponential 
# expansions up to ^2 (quadratic)

variables = ['rot_x', 'rot_y', 'rot_z',
            'trans_x', 'trans_y', 'trans_z']

# testing 
sample_data = basic_confound_df.loc[(basic_confound_df['sub'] == 790) & 
                                   (basic_confound_df['session'] == 2)]

In [82]:
sample_data.head()

,csf,global_signal,rot_x,rot_y,rot_z,run,session,slice,sub,trans_x,trans_y,trans_z,white_matter
55350,12564.621094,9968.958008,0.000097,-0.000320,0.000000,5,2,0,790,-0.000002,-0.029155,0.017870,9685.843750
55351,12254.145508,9911.802734,0.000000,-0.000000,0.000000,5,2,1,790,0.000000,0.022143,0.012066,9672.941406
55352,12090.119141,9892.678711,0.000477,-0.000018,0.000000,5,2,2,790,0.002164,-0.075023,-0.028440,9684.692383
55353,12012.880859,9882.276367,0.000696,-0.000355,0.000089,5,2,3,790,0.000006,-0.061832,-0.012343,9677.000000
55354,11967.224609,9883.166992,0.000000,-0.000417,0.000193,5,2,4,790,0.000025,0.061258,-0.002754,9670.416016


In [83]:
test_var_deriv, test_dat_deriv = temporal_derivatives(order = temporal_derivatives_order, 
                                                variables = variables, 
                                                data = sample_data)

In [84]:
test_deriv_dat.head()

,white_matter,csf,global_signal,rot_x,rot_y,rot_z,trans_x,trans_y,trans_z,white_matter_derivative1,csf_derivative1,global_signal_derivative1,rot_x_derivative1,rot_y_derivative1,rot_z_derivative1,trans_x_derivative1,trans_y_derivative1,trans_z_derivative1
0,9685.843750,12564.621094,9968.958008,0.000097,-0.000320,0.000000,-0.000002,-0.029155,0.017870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9672.941406,12254.145508,9911.802734,0.000000,-0.000000,0.000000,0.000000,0.022143,0.012066,-12.902344,-310.475586,-57.155273,-0.000097,0.000320,0.000000,0.000002,0.051298,-0.005804
2,9684.692383,12090.119141,9892.678711,0.000477,-0.000018,0.000000,0.002164,-0.075023,-0.028440,11.750977,-164.026367,-19.124023,0.000477,-0.000018,0.000000,0.002164,-0.097166,-0.040506
3,9677.000000,12012.880859,9882.276367,0.000696,-0.000355,0.000089,0.000006,-0.061832,-0.012343,-7.692383,-77.238281,-10.402344,0.000219,-0.000337,0.000089,-0.002157,0.013191,0.016097
4,9670.416016,11967.224609,9883.166992,0.000000,-0.000417,0.000193,0.000025,0.061258,-0.002754,-6.583984,-45.656250,0.890625,-0.000696,-0.000061,0.000104,0.000019,0.123091,0.009589


In [85]:
test_var_exp, test_dat_exp = exponential_terms(order = exponential_expansion_order, 
                                                variables = variables, 
                                                data = sample_data)

In [86]:
test_dat_exp.head()

,rot_x,rot_y,rot_z,trans_x,trans_y,trans_z,rot_x_power2,rot_y_power2,rot_z_power2,trans_x_power2,trans_y_power2,trans_z_power2
0,0.000097,-0.000320,0.000000,-0.000002,-0.029155,0.017870,9.431887e-09,1.022196e-07,0.000000e+00,4.000640e-12,0.000850,0.000319
1,0.000000,-0.000000,0.000000,0.000000,0.022143,0.012066,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000490,0.000146
2,0.000477,-0.000018,0.000000,0.002164,-0.075023,-0.028440,2.272276e-07,3.320486e-10,0.000000e+00,4.681468e-06,0.005628,0.000809
3,0.000696,-0.000355,0.000089,0.000006,-0.061832,-0.012343,4.845427e-07,1.261749e-07,7.967633e-09,3.925762e-11,0.003823,0.000152
4,0.000000,-0.000417,0.000193,0.000025,0.061258,-0.002754,0.000000e+00,1.735222e-07,3.716837e-08,6.390177e-10,0.003753,0.000008


In [109]:
final_dfs = []

for s in confound_df['sub'].unique(): 
    for ses in confound_df['session'].unique():
        for run in confound_df['run'].unique():

            data = confound_df.loc[(confound_df['sub'] == s) & 
                                   (confound_df['session'] == ses) & 
                                  confound_df['run'] == run]

            _, temp_dat_deriv = temporal_derivatives(order = temporal_derivatives_order, 
                                                    variables = variables, 
                                                    data = data)
            
            all_vars = list(temp_dat_deriv.columns) # include derivatives 
            
            _, temp_dat_exp = exponential_terms(order = exponential_expansion_order, 
                                                    variables = all_vars, 
                                                    data = temp_dat_deriv)
            temp_dat_exp[a_comp_cor_cols] = data[a_comp_cor_cols]

            final_dfs.append(temp_dat_exp)

In [110]:
confound_deriv_quad_df = pd.concat(final_dfs)

In [113]:
confound_deriv_quad_df.head()

,rot_x,rot_y,rot_z,trans_x,trans_y,trans_z,rot_x_derivative1,rot_y_derivative1,rot_z_derivative1,trans_x_derivative1,...,rot_z_derivative1_power2,trans_x_derivative1_power2,trans_y_derivative1_power2,trans_z_derivative1_power2,a_comp_cor_00,a_comp_cor_01,a_comp_cor_02,a_comp_cor_03,a_comp_cor_04,a_comp_cor_05
0,-0.000235,-8.894170e-05,0.000000,0.009999,-0.019949,0.032686,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.000093,-0.000000e+00,0.000000,0.000000,0.006128,-0.020904,0.000142,8.894170e-05,0.000000,-0.009999,...,0.000000e+00,0.000100,0.000680,0.002872,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-0.000093,-1.155650e-24,-0.000085,0.006466,0.009448,-0.038227,0.000000,-1.155650e-24,-0.000085,0.006466,...,7.271314e-09,0.000042,0.000011,0.000300,0.021215,0.039827,-0.039051,0.038939,-0.061366,0.015082
3,-0.000137,1.589700e-04,0.000000,0.002176,0.165384,0.009432,-0.000044,1.589700e-04,0.000085,-0.004290,...,7.271314e-09,0.000018,0.024316,0.002271,0.054664,0.023060,-0.002183,0.025691,-0.046342,0.067015
4,0.000025,8.089280e-05,0.000000,0.000855,-0.027810,-0.016503,0.000162,-7.807720e-05,0.000000,-0.001321,...,0.000000e+00,0.000002,0.037324,0.000673,0.013892,0.047422,-0.029474,0.011404,-0.050552,0.074103


In [114]:
confound_deriv_quad_df.to_csv(os.path.join(analysis_home, 'confound_df.csv'), index=False)